In [ ]:
#| default_exp sentinel

### Sentinal S2a

Information on the Sentinel spectral specificatoins can be found at:
https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-2-msi/resolutions/spectral

Sentinal S2a Band Definitions.  `sentinal_s2a` defines the band name, center wavelength and
bandwidth in nanometers.  These are used to generate a class for each band that will hold
values that are computed once, and then used repeatedly.

In [ ]:
#| export
import numpy as np

In [ ]:
#| hide
import RedTideProcessing.habspec            as habspec
import RedTideProcessing.wavelength_cal     as cal
import RedTideProcessing.testing            as testing

In [ ]:
#| export
import RedTideProcessing.habspec_data_class as hsdc

## Functions

In [ ]:
#| export
sentinal_s2a = {
# Band
#         Center
#                Width
  'b1' : [ 442.7, 20 ],
  'b2' : [ 492.7, 65 ],
  'b3' : [ 559.8, 35 ],
  'b4' : [ 664.6, 30 ],
  'b8' : [ 832.8, 105],
  
  'b5' : [ 704.1, 14 ],
  'b6' : [ 740.5, 14 ],
  'b7' : [ 782.8, 19 ],
  '8a' : [ 864.7, 21 ],
}

OLCI Spectral band info.

References: 
- [NASA OLCI Spectral band info.](https://ladsweb.modaps.eosdis.nasa.gov/missions-and-measurements/olci/)
- [The Sentinel User Guide](https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-3-olci/overview/heritage)

In [ ]:
#| export
OLCI_Bands = {
# From: https://ladsweb.modaps.eosdis.nasa.gov/missions-and-measurements/olci/
# and: https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-3-olci/overview/heritage
# Band
#         Center
#                Width
  'Oa1'  : [ 400.00, 15,  'Aerosol correction, improved water constituent retrieval' ],
  'Oa2'  : [ 412.50, 10,  'Yellow substance and detrital pigments (Turbidity).' ],
  'Oa3'  : [ 442.50, 10,  'Chl absorption max biogeochemistry, vegetation' ],
  'Oa4'  : [ 490.00, 10,  'High Chl, other pigments' ],
  'Oa5'  : [ 510.00, 10,  'Chl, sediment, turbidity, red tide.' ],
  'Oa6'  : [ 560.00, 10,  'Chlorophyll reference (Chl minimum)' ],
  'Oa7'  : [ 620.00, 10,  'Sediment loading' ],
  'Oa8'  : [ 665.00, 10,  'Chl (2nd Chl abs. max.), sediment, yellow substance/vegetation' ],
  'Oa9'  : [ 673.75, 7.5, 'For improved fluorescence retrieval and to better account for smile together with the bands 665 and 680 nm' ],
  'Oa10' : [ 681.25, 7.5, 'Chl fluorescence peak, red edge' ],
  'Oa11' : [ 708.75, 10,  'Chl fluorescence baseline, red edge transition.' ],
  'Oa12' : [ 753.75, 7.5, 'O2 absorption/clouds<comma> vegetation' ],
  'Oa13' : [ 761.25 ,2.5, 'O2 absorption band/aerosol corr.' ],
  'Oa14' : [ 764.38 ,3.75,'Atmospheric correction' ],
  'Oa15' : [ 767.50 ,2.5, 'O2A used for cloud top pressure, fluorescence over land.' ],
  'Oa16' : [ 778.75 ,15,  'Atmos. corr./aerosol corr.' ],
  'Oa17' : [ 865.00 ,20 , 'Atmos. corr./aclean word to html convertererosol corr., clouds, pixel co-registration.' ],
  'Oa18' : [ 885.00 ,10 , 'Water vapour absorption reference band. Common reference band with SLST instrument. Vegetation monitoring.' ],
  'Oa19' : [ 900.00 ,10 , 'Water vapour absorption/vegetation monitoring (max. reflectance)'],
  'Oa20' : [ 940.00 ,20 , 'Water vapour absorption, atmos./aerosol corr.'],
  'Oa21' : [ 1020.00,40 , 'Atmos./aerosol corr.'],
}



#| hide
### class Sentinal_s2a Band()

See: [Spectral Resolution](https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-2-msi/resolutions/spectral)

In [ ]:
#| export
class Sentinal_Band:
  '''
  Defines a single Sentinel band data object. 
  See: https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-2-msi/resolutions/spectral
  '''
  header = f'band center_nm width_nm color w_low px_low w_high px_high '
  average = True
  mean    = False
  
  def __init__(self,
               hab_spec_class,
               name:str,        # The Sentinel official wavelength-band name.
               center_nm:float, # Center wavelength in nanometers.
               width_nm:float,  # Bandwidth in nanometers.
               color:str        # Desired color to use for any plots, or graphics.
              ) -> object:      # Class object returned.
    #self.x = hab_spec_class.wavelengths
    self.name      = name
    self.center_nm = center_nm
    self.width_nm  = width_nm
    self.color     = color
    self.w_low     = center_nm - width_nm/2
    self.w_high    = center_nm + width_nm/2
    self.px_low    = hsdc.w_to_pixel( hab_spec_class,  self.w_low )
    self.px_high   = hsdc.w_to_pixel( hab_spec_class,  self.w_high )
    self.pixels    = hsdc.w_range_pixels(hab_spec_class, self.w_low,self.w_high)
    self.y_mean    = 0.0        # This gets updated.
  
  def __str__(self):
    return f'{self.name}      {self.center_nm:6.2f}   {self.width_nm:6.1f}  {self.color}'\
           f'{self.w_low:6.1f}   {self.px_low:4d} {self.w_high:6.1f}    {self.px_high:4d}'
  
  def compute_stats(self, 
                  hab_spectra_class     # Spectra to update y_mean from.
                 ) -> None:   # The stat values are updated in the class.
    '''
    Compute the Sentinel band stats for a given spectra if enabled. To enable or
    disable set Sentinel_Band.average = True | False, or Sentinel_Band.mean = True | False.
    '''
    if self.mean:
      self.y_mean    = hab_spectra_class.raw_y[ self.pixels ].mean()
    else:
      self.y_mean = 0.0
      
    if self.average:
      self.y_average = np.average( hab_spectra_class.raw_y[ self.pixels ] )
    else:
      self.y_average = 0.0

#| hide
### def sentinel_update()

In [ ]:
#| export
def sentinel_update( 
  hab_spectra_class, # Spectra to compute Sentinel band averages on.
  sentinel_bands     # Array of sentinel band classes.       
) -> None:           # sentinel_bands will contain .y_mean afterward.
  '''
  '''
  for s in sentinel_bands:
    sentinel_bands[ s ].compute_stats( hab_spectra_class )

## Testing.

Load a json spectra file for testing.

In [ ]:
spectra_file, spectra_note = testing.select_test_file( 0 )    # Load a spectra file for testing.
print(f'Testing with: {spectra_file}')

Testing with: /mnt/s/2021-0717-HAB-Spec-Tampa-Bay-PIE/2021-0717-HAB-pie/164443/hab_spectra/2021-0717-164700-525866-spec.json


test w_to_pixel.

In [ ]:
hab_spectra = hsdc.HAB_Spectra()     # Create an empty spectra class.
print(hab_spectra.__str__())         # Dump it's values.

self.Calibrated_w=False
self.Xpixels=array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])
self.wavelengths=array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])
self.create_time=datetime.datetime(2023, 7, 25, 14, 32, 10, 705023)
self.pixel0=None
self.wavelength0=None
self.pixel1=None
self.wavelength1=None
json_spectra_file_name: None
Lat                 : None
Lon                 : None
altitde_m           : None
n_saturated         : None
raw_y_min           : None
remove_bias         : None
y_average           : None
DateTime            : None
Exposure            : None
raw_y               : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
summed_rows         : 0



Attempt to calibrate created habspec. 

In [ ]:
rv = habspec.calibrate_using_2_wavelengths(   
  hab_spectra,
  spectra_file,
  pixel0 =  73, wavelength0 = 430.774, 
  pixel1 = 941, wavelength1 = 759.370 
)

In [ ]:
print(hab_spectra.__str__())

self.Calibrated_w=True
self.Xpixels=array([   0,    1,    2, ..., 1277, 1278, 1279])
self.wavelengths=array([403.13862212, 403.51718894, 403.89575576, ..., 886.56845161,
       886.94701843, 887.32558525])
self.create_time=datetime.datetime(2023, 7, 25, 14, 32, 10, 705023)
self.pixel0=73
self.wavelength0=430.774
self.pixel1=941
self.wavelength1=759.37
json_spectra_file_name: /mnt/s/2021-0717-HAB-Spec-Tampa-Bay-PIE/2021-0717-HAB-pie/164443/hab_spectra/2021-0717-164700-525866-spec.json
Lat                 : None
Lon                 : None
altitde_m           : None
n_saturated         : None
raw_y_min           : None
remove_bias         : None
y_average           : None
DateTime            : None
Exposure            : None
raw_y               : [73929 73682 73935 ... 39494 39451 39468]
summed_rows         : 800
Xpixels             : [   0    1    2 ... 1277 1278 1279]
wavelengths         : [403.13862212 403.51718894 403.89575576 ... 886.56845161 886.94701843
 887.32558525]
error_msgs   

Test w_range_pixels() between 500 and 530 nanometers.

In [ ]:
test_blue_band_edge = 500
test_red_band_edge  = 520
test_band_pixels = hsdc.w_range_pixels( hab_spectra, test_blue_band_edge, test_red_band_edge)

A numpy array is returned containing the pixel locations for each wavelength
between `test_blue_band_edge` and `test_red_band_edge`

In [ ]:
display(test_band_pixels)

(array([256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268,
        269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281,
        282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294,
        295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307,
        308]),)

We can compute numpy statistic on the spectra values within this band.  We will compute the mean, average, and
min and max on the test_spectra.

In [ ]:
( hab_spectra.raw_y[ test_band_pixels ].mean(),
  np.average( hab_spectra.raw_y[test_band_pixels] ), 
  hab_spectra.raw_y[test_band_pixels].min(), 
  hab_spectra.raw_y[test_band_pixels].max()
)

(108766.83018867925, 108766.83018867925, 101958, 118690)

In [ ]:
hsdc.w_to_pixel( hab_spectra,  500 )

256

In [ ]:
k = 'b2'
sb = Sentinal_Band( hab_spectra, k, sentinal_s2a[k][0], sentinal_s2a[k][1], 'gray')

In [ ]:
sb.name, sb.center_nm, sb.w_low, sb.w_high, sb.px_low, sb.px_high

('b2', 492.7, 460.2, 525.2, 150, 322)

In [ ]:
hsdc.w_to_pixel(hab_spectra, 492.7)

236

In [ ]:
sentinel_array = {}
#hab_spectra = hsdc.HAB_Spectra()
print(Sentinal_Band.header )
for k in sentinal_s2a.keys():
  sentinel_array[k] = Sentinal_Band( hab_spectra, k, sentinal_s2a[k][0], sentinal_s2a[k][1], 'gray')
  print(sentinel_array[k].__str__() )

band center_nm width_nm color w_low px_low w_high px_high 
b2      492.70     65.0  gray 460.2    150  525.2     322
b3      559.80     35.0  gray 542.3    367  577.3     460
b4      664.60     30.0  gray 649.6    651  679.6     730
b8      832.80    105.0  gray 780.3    997  885.3    1274
b5      704.10     14.0  gray 697.1    777  711.1     814
b6      740.50     14.0  gray 733.5    873  747.5     910
b7      782.80     19.0  gray 773.3    978  792.3    1028
8a      864.70     21.0  gray 854.2   1192  875.2    1247


In [ ]:
sentinel_array['b2'].__str__()

'b2      492.70     65.0  gray 460.2    150  525.2     322'

In [ ]:
sentinel_array['b3'].__str__()

'b3      559.80     35.0  gray 542.3    367  577.3     460'

In [ ]:
sentinel_array['b3'].center_nm

559.8

In [ ]:
band = sentinel_array['b3'].pixels
band

(array([368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380,
        381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393,
        394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406,
        407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419,
        420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432,
        433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445,
        446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458,
        459, 460]),)

In [ ]:
###### test_spectra[ band ].mean()

In [ ]:
## %%timeit
sentinel_update( hab_spectra, sentinel_array )

In [ ]:
print('Band    y_mean   y_average')
for se in sentinel_array:
  sentinel_array[se].compute_stats( hab_spectra )  
  print(f'  {se}   {sentinel_array[se].y_mean:7.3f}  {sentinel_array[se].y_average:7.3f}')
  #print(y_avg)

Band    y_mean   y_average
  b2     0.000  106653.831
  b3     0.000  181290.839
  b4     0.000  72964.013
  b8     0.000  40665.805
  b5     0.000  87887.892
  b6     0.000  45640.946
  b7     0.000  42748.780
  8a     0.000  39435.964
